In [1]:
from google.colab import drive
import os

# Montar Google Drive
drive.mount('/content/drive', force_remount=True)
folder_path = '/content/drive/MyDrive/Google Maps'


# Recorrer y mostrar archivos
for root, dirs, files in os.walk(folder_path):
    print(f'📂 Carpeta: {root}')
    for file in files:
        print(f'  📄 {file}')


Mounted at /content/drive
📂 Carpeta: /content/drive/MyDrive/Google Maps
  📄 Diccionario de datos.gdoc
📂 Carpeta: /content/drive/MyDrive/Google Maps/reviews-estados
📂 Carpeta: /content/drive/MyDrive/Google Maps/reviews-estados/review-Alabama
  📄 1.json
  📄 2.json
  📄 3.json
  📄 4.json
  📄 5.json
  📄 6.json
  📄 7.json
  📄 8.json
  📄 9.json
  📄 10.json
  📄 11.json
  📄 12.json
📂 Carpeta: /content/drive/MyDrive/Google Maps/reviews-estados/review-Connecticut
  📄 1.json
  📄 2.json
  📄 3.json
  📄 4.json
  📄 5.json
  📄 6.json
  📄 7.json
  📄 8.json
  📄 9.json
  📄 10.json
  📄 11.json
  📄 12.json
  📄 13.json
  📄 14.json
  📄 15.json
  📄 16.json
  📄 17.json
  📄 18.json
📂 Carpeta: /content/drive/MyDrive/Google Maps/reviews-estados/review-California
  📄 1.json
  📄 2.json
  📄 3.json
  📄 4.json
  📄 5.json
  📄 6.json
  📄 7.json
  📄 8.json
  📄 9.json
  📄 10.json
  📄 11.json
  📄 12.json
  📄 13.json
  📄 14.json
  📄 15.json
  📄 16.json
  📄 17.json
  📄 18.json
📂 Carpeta: /content/drive/MyDrive/Google Maps/rev

In [3]:
file_path = '/content/drive/MyDrive/Google Maps/metadata-sitios/1.json'

In [ ]:
with open(file_path, 'r', encoding='utf-8') as f:
    # Leer solo los primeros 1000 caracteres
    snippet = f.read(10000)
    print(snippet)

{"name": "Porter Pharmacy", "address": "Porter Pharmacy, 129 N Second St, Cochran, GA 31014", "gmap_id": "0x88f16e41928ff687:0x883dad4fd048e8f8", "description": null, "latitude": 32.3883, "longitude": -83.3571, "category": ["Pharmacy"], "avg_rating": 4.9, "num_of_reviews": 16, "price": null, "hours": [["Friday", "8AM\u20136PM"], ["Saturday", "8AM\u201312PM"], ["Sunday", "Closed"], ["Monday", "8AM\u20136PM"], ["Tuesday", "8AM\u20136PM"], ["Wednesday", "8AM\u201312PM"], ["Thursday", "8AM\u20136PM"]], "MISC": {"Service options": ["In-store shopping", "Same-day delivery"], "Health & safety": ["Mask required", "Staff required to disinfect surfaces between visits"], "Accessibility": ["Wheelchair accessible entrance"], "Planning": ["Quick visit"]}, "state": "Open \u22c5 Closes 6PM", "relative_results": ["0x88f16e41929435cf:0x5b2532a2885e9ef6", "0x88f16c32716531c1:0x5f19bdaa5044e4fa", "0x88f16e6e3f4a21df:0xcf495da9bb4d89ea"], "url": "https://www.google.com/maps/place//data=!4m2!3m1!1s0x88f16e4

## Combinemos todos los json en un solo archivo

In [2]:
import os
import json
import pandas as pd
import glob

In [ ]:
carpeta_json = '/content/drive/MyDrive/Google Maps/metadata-sitios'
salida_dir = '/content/drive/MyDrive/Descargados de Colab/parquets_chunks'

In [ ]:
def aplanar_json(datos, prefijo=""):
    resultado = {}
    for clave, valor in datos.items():
        nueva_clave = f"{prefijo}{clave}"
        if isinstance(valor, dict):
            resultado.update(aplanar_json(valor, f"{nueva_clave}_"))
        elif isinstance(valor, list):
            for i, item in enumerate(valor):
                if isinstance(item, (dict, list)):
                    resultado.update(aplanar_json({str(i): item}, f"{nueva_clave}_"))
                else:
                    resultado[f"{nueva_clave}_{i}"] = item
        else:
            resultado[nueva_clave] = valor
    return resultado

# ===== CONFIGURACIÓN =====
os.makedirs(salida_dir, exist_ok=True)

chunk_size = 10000  # Cantidad de registros por chunk
buffer = []
chunk_id = 0

# Buscar archivos
archivos_json = glob.glob(os.path.join(carpeta_json, '*.json'))
print(f"Se encontraron {len(archivos_json)} archivos.")

for ruta_json in archivos_json:
    print(f"Procesando archivo: {os.path.basename(ruta_json)}")
    with open(ruta_json, 'r', encoding='utf-8') as f:
        for linea in f:
            try:
                data = json.loads(linea)
                data_flat = aplanar_json(data)
                buffer.append(data_flat)

                if len(buffer) >= chunk_size:
                    df_chunk = pd.DataFrame(buffer)
                    ruta_parquet = os.path.join(salida_dir, f"chunk_{chunk_id}.parquet")
                    df_chunk.to_parquet(ruta_parquet, index=False)
                    print(f"💾 Chunk {chunk_id} guardado con {len(buffer)} registros.")
                    buffer = []
                    chunk_id += 1
            except Exception as e:
                print(f"⚠️ Error leyendo línea: {e}")

# Guardar lo que quedó en el buffer
if buffer:
    df_chunk = pd.DataFrame(buffer)
    ruta_parquet = os.path.join(salida_dir, f"chunk_{chunk_id}.parquet")
    df_chunk.to_parquet(ruta_parquet, index=False)
    print(f"💾 Último chunk {chunk_id} guardado con {len(buffer)} registros.")


Se encontraron 0 archivos.


In [ ]:
import pyarrow.parquet as pq
import pyarrow as pa

# Leer todos los chunks y unirlos
parquet_files = glob.glob(os.path.join(salida_dir, 'chunk_*.parquet'))
dfs = [pd.read_parquet(pq_file) for pq_file in parquet_files]

df_final = pd.concat(dfs, ignore_index=True)
df_final.to_parquet(os.path.join(salida_dir, 'unificado.parquet'), index=False)
print("✅ Archivo final unido guardado.")


In [ ]:
import os
import glob
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd

parquet_files = glob.glob(os.path.join(salida_dir, 'chunk_*.parquet'))

# Leer el primer archivo y obtener las columnas base
df_base = pd.read_parquet(parquet_files[0])
columnas_base = df_base.columns
schema = pa.Table.from_pandas(df_base).schema

output_path = os.path.join(salida_dir, 'aa_unificado.parquet')

with pq.ParquetWriter(output_path, schema) as writer:
    for pq_file in parquet_files:
        df = pd.read_parquet(pq_file)
        # Reindex para que todas tengan las mismas columnas, rellenando con NaN si falta alguna
        df = df.reindex(columns=columnas_base)
        table = pa.Table.from_pandas(df, schema=schema)
        writer.write_table(table)
        print(f"✅ Escrito {pq_file}")

print("✅ Archivo final unido guardado.")


✅ Escrito /content/drive/MyDrive/Descargados de Colab/parquets_chunks/chunk_0.parquet
✅ Escrito /content/drive/MyDrive/Descargados de Colab/parquets_chunks/chunk_1.parquet
✅ Escrito /content/drive/MyDrive/Descargados de Colab/parquets_chunks/chunk_2.parquet
✅ Escrito /content/drive/MyDrive/Descargados de Colab/parquets_chunks/chunk_3.parquet
✅ Escrito /content/drive/MyDrive/Descargados de Colab/parquets_chunks/chunk_4.parquet
✅ Escrito /content/drive/MyDrive/Descargados de Colab/parquets_chunks/chunk_5.parquet
✅ Escrito /content/drive/MyDrive/Descargados de Colab/parquets_chunks/chunk_6.parquet
✅ Escrito /content/drive/MyDrive/Descargados de Colab/parquets_chunks/chunk_7.parquet
✅ Escrito /content/drive/MyDrive/Descargados de Colab/parquets_chunks/chunk_8.parquet
✅ Escrito /content/drive/MyDrive/Descargados de Colab/parquets_chunks/chunk_9.parquet
✅ Escrito /content/drive/MyDrive/Descargados de Colab/parquets_chunks/chunk_10.parquet
✅ Escrito /content/drive/MyDrive/Descargados de Colab

In [ ]:
import pandas as pd

# Ruta del archivo unificado
archivo_unificado = '/content/drive/MyDrive/Descargados de Colab/parquets_chunks/unificado.parquet'

# Cargar el parquet
df = pd.read_parquet(archivo_unificado)

# Resumen general
print("📊 Resumen del archivo unificado:\n")
print(f"✅ Filas: {df.shape[0]:,}")
print(f"✅ Columnas: {df.shape[1]:,}\n")

# Tipos de datos
print("🧬 Tipos de datos por columna:")
print(df.dtypes)

# Nulos por columna
print("\n🕳️ Cantidad de valores nulos por columna:")
print(df.isnull().sum())

# Porcentaje de nulos
print("\n📉 Porcentaje de nulos por columna:")
print((df.isnull().mean() * 100).round(2).astype(str) + '%')

# Estadísticas generales (solo numéricas)
print("\n📐 Estadísticas descriptivas:")
print(df.describe())


📊 Resumen del archivo unificado:

✅ Filas: 3,025,011
✅ Columnas: 115

🧬 Tipos de datos por columna:
name                         object
address                      object
gmap_id                      object
description                  object
latitude                    float64
                             ...   
MISC_Offerings_11            object
MISC_Atmosphere_2            object
MISC_From the business_2     object
MISC_Dining options_5        object
MISC_Health & safety_6       object
Length: 115, dtype: object

🕳️ Cantidad de valores nulos por columna:
name                             37
address                       80511
gmap_id                           0
description                 2770722
latitude                          0
                             ...   
MISC_Offerings_11           3019672
MISC_Atmosphere_2           3018879
MISC_From the business_2    3024302
MISC_Dining options_5       3020475
MISC_Health & safety_6      3024686
Length: 115, dtype: int64

📉 Porcentaj

## Ahora trabajamos con los reviews de los estados


In [2]:
carpeta_json = '/content/drive/MyDrive/Google Maps/reviews-estados'
salida_dir = '/content/drive/MyDrive/Descargados de Colab/parquets_chunks_reviews'

In [ ]:
import os
import glob
import json
import pandas as pd

def aplanar_json(datos, prefijo=""):
    resultado = {}
    for clave, valor in datos.items():
        nueva_clave = f"{prefijo}{clave}"
        if isinstance(valor, dict):
            resultado.update(aplanar_json(valor, f"{nueva_clave}_"))
        elif isinstance(valor, list):
            for i, item in enumerate(valor):
                if isinstance(item, (dict, list)):
                    resultado.update(aplanar_json({str(i): item}, f"{nueva_clave}_"))
                else:
                    resultado[f"{nueva_clave}_{i}"] = item
        else:
            resultado[nueva_clave] = valor
    return resultado

# ===== CONFIGURACIÓN =====
os.makedirs(salida_dir, exist_ok=True)

chunk_size = 10000
buffer = []
chunk_id = 0

# 🔍 Buscar todos los archivos .json en la carpeta y subcarpetas
archivos_json = glob.glob(os.path.join(carpeta_json, '**', '*.json'), recursive=True)
print(f"📁 Se encontraron {len(archivos_json)} archivos JSON en todas las subcarpetas.")

# Procesar cada archivo
for ruta_json in archivos_json:
    # Extraer el nombre del estado (sin 'review-')
    nombre_carpeta = os.path.basename(os.path.dirname(ruta_json))
    estado = nombre_carpeta.replace('review-', '')

    print(f"📄 Procesando archivo: {ruta_json} (Estado: {estado})")
    try:
        with open(ruta_json, 'r', encoding='utf-8') as f:
            for linea in f:
                try:
                    data = json.loads(linea)
                    data_flat = aplanar_json(data)
                    data_flat['estado'] = estado  # ➕ Agregar la columna estado
                    buffer.append(data_flat)

                    if len(buffer) >= chunk_size:
                        df_chunk = pd.DataFrame(buffer)
                        ruta_parquet = os.path.join(salida_dir, f"chunk_{chunk_id}.parquet")
                        df_chunk.to_parquet(ruta_parquet, index=False)
                        print(f"💾 Chunk {chunk_id} guardado con {len(buffer)} registros.")
                        buffer = []
                        chunk_id += 1
                except Exception as e:
                    print(f"⚠️ Error leyendo línea en {ruta_json}: {e}")
    except Exception as e:
        print(f"❌ Error abriendo archivo {ruta_json}: {e}")

# Guardar lo que quedó
if buffer:
    df_chunk = pd.DataFrame(buffer)
    ruta_parquet = os.path.join(salida_dir, f"chunk_{chunk_id}.parquet")
    df_chunk.to_parquet(ruta_parquet, index=False)
    print(f"💾 Último chunk {chunk_id} guardado con {len(buffer)} registros.")


Se truncaron las últimas líneas 5000 del resultado de transmisión.
💾 Chunk 4313 guardado con 10000 registros.
💾 Chunk 4314 guardado con 10000 registros.
💾 Chunk 4315 guardado con 10000 registros.
💾 Chunk 4316 guardado con 10000 registros.
💾 Chunk 4317 guardado con 10000 registros.
💾 Chunk 4318 guardado con 10000 registros.
💾 Chunk 4319 guardado con 10000 registros.
💾 Chunk 4320 guardado con 10000 registros.
💾 Chunk 4321 guardado con 10000 registros.
💾 Chunk 4322 guardado con 10000 registros.
💾 Chunk 4323 guardado con 10000 registros.
💾 Chunk 4324 guardado con 10000 registros.
💾 Chunk 4325 guardado con 10000 registros.
💾 Chunk 4326 guardado con 10000 registros.
💾 Chunk 4327 guardado con 10000 registros.
📄 Procesando archivo: /content/drive/MyDrive/Google Maps/reviews-estados/review-Massachusetts/11.json (Estado: Massachusetts)
💾 Chunk 4328 guardado con 10000 registros.
💾 Chunk 4329 guardado con 10000 registros.
💾 Chunk 4330 guardado con 10000 registros.
💾 Chunk 4331 guardado con 10000 r

In [1]:
import os
import glob
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd

parquet_files = glob.glob(os.path.join(salida_dir, 'chunk_*.parquet'))

# Leer el primer archivo y obtener las columnas base
df_base = pd.read_parquet(parquet_files[0])
columnas_base = df_base.columns
schema = pa.Table.from_pandas(df_base).schema

output_path = os.path.join(salida_dir, 'aa_unificado.parquet')

with pq.ParquetWriter(output_path, schema) as writer:
    for pq_file in parquet_files:
        df = pd.read_parquet(pq_file)
        # Reindex para que todas tengan las mismas columnas, rellenando con NaN si falta alguna
        df = df.reindex(columns=columnas_base)
        table = pa.Table.from_pandas(df, schema=schema)
        writer.write_table(table)
        print(f"✅ Escrito {pq_file}")

print("✅ Archivo final unido guardado.")


NameError: name 'salida_dir' is not defined

In [3]:
import dask.dataframe as dd

# Carga sin meterlo todo en RAM
ddf = dd.read_parquet('/content/drive/MyDrive/Descargados de Colab/parquets_chunks_reviews/aa_unificado.parquet')

# Solo las primeras filas
df_sample = ddf.head(1000)  # Esto sí lo carga a pandas, pero solo esas filas

# Verifica
print(df_sample.shape)
print(df_sample.head())


(1000, 42)
                 user_id                     name           time  rating  \
0  113766065560016413369           TheDiamondDave  1617676778946       5   
1  116151043210713142064           Kathy Marshall  1612038163606       5   
2  116088685202086458845          Teresa Ferguson  1602962017110       5   
3  103082764991629671969  Country Strong Outdoors  1612852861583       4   
4  106923743092573625151            Sierra Curtis  1610936215696       5   

                                                text  pics  resp  \
0  Good supply of products friendly staff ! Close...   NaN   NaN   
1      Awesome store so very helpful. Thanks so much   NaN   NaN   
2  Opening weekend for the new location! I am so ...   NaN   NaN   
3  Love Tractor supply but not crazy about the wa...   NaN   NaN   
4  We go here every two weeks to pick up supplies...   NaN   NaN   

                                gmap_id          estado pics_0_url_0  ...  \
0  0x88584f6e2704ff77:0x5ed0a4b5967575a  South